In [3]:
!pip uninstall -y pyarrow datasets
!pip install --no-use-pep517 pyarrow
!pip install datasets
!pip install torch transformers
!pip install transformers[torch]
!pip install accelerate -U
!pip install scikit-learn

Found existing installation: pyarrow 16.0.0
Uninstalling pyarrow-16.0.0:
  Successfully uninstalled pyarrow-16.0.0
Found existing installation: datasets 2.19.0
Uninstalling datasets-2.19.0:
  Successfully uninstalled datasets-2.19.0
  Using cached pyarrow-16.0.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (3.0 kB)
Using cached pyarrow-16.0.0-cp311-cp311-manylinux_2_28_x86_64.whl (40.8 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlflow 2.11.3 requires pyarrow<16,>=4.0.0, but you have pyarrow 16.0.0 which is incompatible.
  Using cached datasets-2.19.0-py3-none-any.whl.metadata (19 kB)
Using cached datasets-2.19.0-py3-none-any.whl (542 kB)


In [19]:
from datasets import Dataset, DatasetDict, load_dataset
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [20]:

#Load dataset
dataset = load_dataset("Hello-SimpleAI/HC3", name="all")
human_answers = dataset['train']['human_answers']
chatgpt_answers = dataset['train']['chatgpt_answers']
human_answers = [string for sublist in dataset['train']['human_answers'] for string in sublist if sublist]
chatgpt_answers = [string for sublist in dataset['train']['chatgpt_answers'] for string in sublist if sublist]
data_dict = {
    "texts": human_answers + chatgpt_answers,
    "labels": [1] * len(human_answers) + [0] * len(chatgpt_answers)
}
hf_dataset = Dataset.from_dict(data_dict)
texts = hf_dataset["texts"]
labels = hf_dataset["labels"]

# Vectorize
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, np.array(labels), test_size=0.2, random_state=42)


In [21]:


# Initialize the modl
model = LogisticRegression()

# Train it
model.fit(X_train, y_train)
y_pred = model.predict(X_test)


/opt/mamba/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [22]:


# Results
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)
print("Matrice de confusion:\n", conf_matrix)
print("\nRapport de classification:\n", class_report)

Matrice de confusion:
 [[ 5104   234]
 [  168 11584]]

Rapport de classification:
               precision    recall  f1-score   support

           0       0.97      0.96      0.96      5338
           1       0.98      0.99      0.98     11752

    accuracy                           0.98     17090
   macro avg       0.97      0.97      0.97     17090
weighted avg       0.98      0.98      0.98     17090

